In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

sns.set_theme(style='darkgrid')
from datetime import datetime as dt

from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, classification_report

import sys

sys.path.insert(1,'lib/python3.10/site-packages')

import duckdb
import optuna

/tmp/ipykernel_3481/2347561443.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from sklearn.metrics import roc_auc_score

In [3]:
df = pd.read_csv('truelayer_data_sc_test_data_set.csv')

In [4]:
df

,id,bank_id,currency,status,api_version,failure_reason,failure_stage,customer_id,vertical,connectivity_type,amount_in_currency,country_id,createdat_ts,lastupdatedat_ts,initiated_at,executed_at,failed_at,authorizing_at,authorized_at,settled_at
0,3c6646ef17a52ac2e35fdb6a15aa44db6d85985b,b5ab8f51d35b64af079485e5bbbc335213f3a0ae,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,28.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2010-02-10 06:00:06.452249,2010-02-10 06:00:14.766862,2010-02-10 06:00:06.452249,2010-02-10 06:00:14.766862,NaN,2010-02-10 06:00:06.452249,2010-02-10 06:00:06.452249,NaN
1,6c3e58befae5a4880c62c54bdffe7243ab7b66cc,34a0061ba48d1c2810cd930dd69d42482ca92d85,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,18.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2010-01-12 06:00:17.890703,2010-01-12 06:00:22.517782,2010-01-12 06:00:17.890703,2010-01-12 06:00:22.517782,NaN,2010-01-12 06:00:17.890703,2010-01-12 06:00:17.890703,NaN
2,b2ee452081f287aa12f0efbd05c278edd332def9,660054a4565377c6e43ff7709abf56d8494ba604,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,28.5,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2010-02-10 07:20:57.816799,2010-02-10 07:21:04.284348,2010-02-10 07:20:57.816799,2010-02-10 07:21:04.284348,NaN,2010-02-10 07:20:57.816799,2010-02-10 07:20:57.816799,NaN
3,5aa68c655e0ca1e57bcf4a70c269151811e3ee77,660054a4565377c6e43ff7709abf56d8494ba604,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,16.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2010-02-10 06:00:20.172570,2010-02-10 06:00:25.549863,2010-02-10 06:00:20.172570,2010-02-10 06:00:25.549863,NaN,2010-02-10 06:00:20.172570,2010-02-10 06:00:20.172570,NaN
4,1d72b60adefab1066d7421d72a2c3abb3d558544,0090e57640ed78b0c16ac4606a6773769545bb17,CAD,Executed,v3,NaN,NaN,64c370a753b1db282770cdbf2aba5434a3185b3d,vertical 2,type 8,5.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2009-08-14 13:42:02.939891,2009-08-14 13:42:54.515245,2009-08-14 13:42:09.978416,2009-08-14 13:42:54.515245,NaN,2009-08-14 13:42:03.699860,2009-08-14 13:42:52.858713,2009-08-14 13:42:54.515245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,920984344ede2b361965b4282c8d65ad1f35a949,6926610db1dfc17aca46a6f67789d7d6c24a7185,CAD,Executed,v3,NaN,NaN,63237c0742e42bfd7d59d68d894d010aa4cca170,vertical 2,type 8,10.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2009-06-23 13:36:27.430381,2009-06-23 13:36:33.607752,2009-06-23 13:36:27.430381,2009-06-23 13:36:33.607752,NaN,2009-06-23 13:36:27.430381,2009-06-23 13:36:27.430381,NaN
499996,a287bf9507414e9afd007f9387c39001d364b5ba,34a0061ba48d1c2810cd930dd69d42482ca92d85,CAD,Executed,v3,NaN,NaN,63237c0742e42bfd7d59d68d894d010aa4cca170,vertical 2,type 8,10.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2009-03-30 15:20:17.890930,2009-03-30 15:20:46.035369,2009-03-30 15:20:18.787167,2009-03-30 15:20:46.035369,NaN,2009-03-30 15:20:18.787167,2009-03-30 15:20:43.573008,NaN
499997,5d228fb6478545b2e1082cb91d1f2e867f88ef88,a40fe54e8fd65d2ce458adba016a5a23784c3647,USD,Executed,v3,NaN,NaN,63237c0742e42bfd7d59d68d894d010aa4cca170,vertical 2,type 8,22.0,7ec009376b8bf0c03561b3f6ba251ab9d96efd42,2009-11-28 06:02:28.703718,2009-11-28 06:03:23.918508,2009-11-28 06:02:29.500681,2009-11-28 06:03:23.918508,NaN,2009-11-28 06:02:29.500681,2009-11-28 06:03:23.916513,NaN
499998,9fec71dfcae8f82430647068998e943673faad12,6c4968fa85cc234c89763375210a95a9d789e501,USD,Executed,v2,NaN,NaN,63237c0742e42bfd7d59d68d894d010aa4cca170,vertical 2,NaN,500.0,f69ad584c025f9b756ce1e8c70de3a33aef50b32,2009-02-19 22:36:54.932748,2009-02-22 08:54:14.384793,NaN,2009-02-22 08:54:14.384793,NaN,NaN,NaN,NaN


In [7]:
df = df.loc[(df['status']=='Executed') | (df['status']=='Failed') | (df['status']=='failed')]

In [8]:
df['status'].value_counts()

status
Executed    377616
failed       44621
Failed         478
Name: count, dtype: int64

In [9]:
df['status'] = df['status'].apply(lambda x: 1 if x.lower() != 'failed' else 0)

/tmp/ipykernel_9021/600294653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['status'] = df['status'].apply(lambda x: 1 if x.lower() != 'failed' else 0)


In [10]:
df['status'].value_counts()

status
1    377616
0     45099
Name: count, dtype: int64

In [13]:
df.columns

Index(['id', 'bank_id', 'currency', 'status', 'api_version', 'failure_reason',
       'failure_stage', 'customer_id', 'vertical', 'connectivity_type',
       'amount_in_currency', 'country_id', 'createdat_ts', 'lastupdatedat_ts',
       'initiated_at', 'executed_at', 'failed_at', 'authorizing_at',
       'authorized_at', 'settled_at', 'total_duration', 'authorizing_duration',
       'authorized_duration'],
      dtype='object')

In [14]:
drop = ['id','failure_reason','failure_stage','customer_id','createdat_ts','lastupdatedat_ts','initiated_at','executed_at',
        'failed_at','authorizing_at','authorized_at','settled_at']
df.drop(drop, axis=1,inplace=False)

,bank_id,currency,status,api_version,vertical,connectivity_type,amount_in_currency,country_id,total_duration,authorizing_duration,authorized_duration
0,b5ab8f51d35b64af079485e5bbbc335213f3a0ae,CAD,1,v3,vertical 2,type 8,28.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,8,0.0,0.0
1,34a0061ba48d1c2810cd930dd69d42482ca92d85,CAD,1,v3,vertical 2,type 8,18.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,4,0.0,0.0
2,660054a4565377c6e43ff7709abf56d8494ba604,CAD,1,v3,vertical 2,type 8,28.5,9e2b06736b477b7a924f60de14a7e329d82d6f4f,6,0.0,0.0
3,660054a4565377c6e43ff7709abf56d8494ba604,CAD,1,v3,vertical 2,type 8,16.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,5,0.0,0.0
4,0090e57640ed78b0c16ac4606a6773769545bb17,CAD,1,v3,vertical 2,type 8,5.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,51,0.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...
499995,6926610db1dfc17aca46a6f67789d7d6c24a7185,CAD,1,v3,vertical 2,type 8,10.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,6,0.0,0.0
499996,34a0061ba48d1c2810cd930dd69d42482ca92d85,CAD,1,v3,vertical 2,type 8,10.0,9e2b06736b477b7a924f60de14a7e329d82d6f4f,28,0.0,24.0
499997,a40fe54e8fd65d2ce458adba016a5a23784c3647,USD,1,v3,vertical 2,type 8,22.0,7ec009376b8bf0c03561b3f6ba251ab9d96efd42,55,0.0,54.0
499998,6c4968fa85cc234c89763375210a95a9d789e501,USD,1,v2,vertical 2,NaN,500.0,f69ad584c025f9b756ce1e8c70de3a33aef50b32,37039,NaN,NaN


In [15]:
features = ['bank_id', 'currency', 'api_version', 'vertical', 'connectivity_type',
       'amount_in_currency', 'country_id',  'total_duration', 'authorizing_duration',
       'authorized_duration']

label = ['status']

In [16]:
X = df[features].copy()
y = df[label].values.ravel()

In [18]:
y.shape

(422715,)

In [8]:
def preprocess(df):
    df['total_duration'] = df.lastupdatedat_ts.apply(lambda x: dt.fromisoformat(x)) - df.createdat_ts.apply(lambda x: dt.fromisoformat(x))
    df['total_duration'] = df['total_duration'].apply(lambda delta: delta.seconds)
    
    df['authorizing_duration'] = df.authorizing_at.apply(lambda x:  x if pd.isnull(x) else dt.fromisoformat(x)) - df.createdat_ts.apply(lambda x: dt.fromisoformat(x))
    df['authorizing_duration'] = df['authorizing_duration'].apply(lambda delta: delta if pd.isnull(delta) else delta.seconds)
    df['authorizing_duration'].fillna(np.nan, inplace=True)
    
    df['authorized_duration'] = df.authorized_at.apply(lambda x: x if pd.isnull(x) else dt.fromisoformat(x)) - df.authorizing_at.apply(lambda x: x if pd.isnull(x) else dt.fromisoformat(x))
    df['authorized_duration'] = df['authorized_duration'].apply(lambda delta: delta if pd.isnull(delta) else delta.seconds)
    df['authorized_duration'].fillna(np.nan, inplace=True)


    categories = ['bank_id', 'currency','api_version', 'vertical', 'connectivity_type',
       'country_id']

    numerical = ['amount_in_currency','total_duration', 'authorizing_duration',
           'authorized_duration']
    
    for col in categories:
        df[col] = df[col].astype('category')
    
    for col in numerical:
        df[col] = df[col].astype('float')

    return df

In [9]:
processed = preprocess(df)
processed

/tmp/ipykernel_3481/1390526987.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['authorizing_duration'].fillna(np.nan, inplace=True)
/tmp/ipykernel_3481/1390526987.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['authorizing_duration'].fillna(np.nan, inplace=True)
/tmp/ipykernel_3481/1390526987.py:

,id,bank_id,currency,status,api_version,failure_reason,failure_stage,customer_id,vertical,connectivity_type,...,lastupdatedat_ts,initiated_at,executed_at,failed_at,authorizing_at,authorized_at,settled_at,total_duration,authorizing_duration,authorized_duration
0,3c6646ef17a52ac2e35fdb6a15aa44db6d85985b,b5ab8f51d35b64af079485e5bbbc335213f3a0ae,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,...,2010-02-10 06:00:14.766862,2010-02-10 06:00:06.452249,2010-02-10 06:00:14.766862,NaN,2010-02-10 06:00:06.452249,2010-02-10 06:00:06.452249,NaN,8.0,0.0,0.0
1,6c3e58befae5a4880c62c54bdffe7243ab7b66cc,34a0061ba48d1c2810cd930dd69d42482ca92d85,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,...,2010-01-12 06:00:22.517782,2010-01-12 06:00:17.890703,2010-01-12 06:00:22.517782,NaN,2010-01-12 06:00:17.890703,2010-01-12 06:00:17.890703,NaN,4.0,0.0,0.0
2,b2ee452081f287aa12f0efbd05c278edd332def9,660054a4565377c6e43ff7709abf56d8494ba604,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,...,2010-02-10 07:21:04.284348,2010-02-10 07:20:57.816799,2010-02-10 07:21:04.284348,NaN,2010-02-10 07:20:57.816799,2010-02-10 07:20:57.816799,NaN,6.0,0.0,0.0
3,5aa68c655e0ca1e57bcf4a70c269151811e3ee77,660054a4565377c6e43ff7709abf56d8494ba604,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,...,2010-02-10 06:00:25.549863,2010-02-10 06:00:20.172570,2010-02-10 06:00:25.549863,NaN,2010-02-10 06:00:20.172570,2010-02-10 06:00:20.172570,NaN,5.0,0.0,0.0
4,1d72b60adefab1066d7421d72a2c3abb3d558544,0090e57640ed78b0c16ac4606a6773769545bb17,CAD,Executed,v3,NaN,NaN,64c370a753b1db282770cdbf2aba5434a3185b3d,vertical 2,type 8,...,2009-08-14 13:42:54.515245,2009-08-14 13:42:09.978416,2009-08-14 13:42:54.515245,NaN,2009-08-14 13:42:03.699860,2009-08-14 13:42:52.858713,2009-08-14 13:42:54.515245,51.0,0.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,920984344ede2b361965b4282c8d65ad1f35a949,6926610db1dfc17aca46a6f67789d7d6c24a7185,CAD,Executed,v3,NaN,NaN,63237c0742e42bfd7d59d68d894d010aa4cca170,vertical 2,type 8,...,2009-06-23 13:36:33.607752,2009-06-23 13:36:27.430381,2009-06-23 13:36:33.607752,NaN,2009-06-23 13:36:27.430381,2009-06-23 13:36:27.430381,NaN,6.0,0.0,0.0
499996,a287bf9507414e9afd007f9387c39001d364b5ba,34a0061ba48d1c2810cd930dd69d42482ca92d85,CAD,Executed,v3,NaN,NaN,63237c0742e42bfd7d59d68d894d010aa4cca170,vertical 2,type 8,...,2009-03-30 15:20:46.035369,2009-03-30 15:20:18.787167,2009-03-30 15:20:46.035369,NaN,2009-03-30 15:20:18.787167,2009-03-30 15:20:43.573008,NaN,28.0,0.0,24.0
499997,5d228fb6478545b2e1082cb91d1f2e867f88ef88,a40fe54e8fd65d2ce458adba016a5a23784c3647,USD,Executed,v3,NaN,NaN,63237c0742e42bfd7d59d68d894d010aa4cca170,vertical 2,type 8,...,2009-11-28 06:03:23.918508,2009-11-28 06:02:29.500681,2009-11-28 06:03:23.918508,NaN,2009-11-28 06:02:29.500681,2009-11-28 06:03:23.916513,NaN,55.0,0.0,54.0
499998,9fec71dfcae8f82430647068998e943673faad12,6c4968fa85cc234c89763375210a95a9d789e501,USD,Executed,v2,NaN,NaN,63237c0742e42bfd7d59d68d894d010aa4cca170,vertical 2,NaN,...,2009-02-22 08:54:14.384793,NaN,2009-02-22 08:54:14.384793,NaN,NaN,NaN,NaN,37039.0,NaN,NaN


In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, test_size=.2, random_state=42)

In [87]:
model = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=.1, enable_categorical=True)
model.fit(X_train_, y_train_)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [88]:
y_pred = model.predict(X_test_)
roc = roc_auc_score(y_test_, y_pred)

In [89]:
roc

0.9537803005972847

In [90]:
model.feature_importances_ * 100

array([ 1.8044475 ,  0.7938415 , 18.868233  ,  0.5305319 ,  3.7687466 ,
        0.6239432 ,  0.35084406, 32.077526  ,  0.58922386, 40.592663  ],
      dtype=float32)

In [91]:
features

['bank_id',
 'currency',
 'api_version',
 'vertical',
 'connectivity_type',
 'amount_in_currency',
 'country_id',
 'total_duration',
 'authorizing_duration',
 'authorized_duration']

In [94]:
feature_mask = model.feature_importances_ * 100 >= 1

In [95]:
feature_mask

array([ True, False,  True, False,  True, False, False,  True, False,
        True])

In [96]:
y_train

array([1, 0, 1, ..., 1, 1, 1])

In [97]:
X_train = X_train.iloc[:,feature_mask]
X_test = X_test.iloc[:, feature_mask]
X_train_ = X_train_.iloc[:,feature_mask]
X_test_ = X_test_.iloc[:, feature_mask]

In [98]:
def objective(trial):
    params = {
        "verbosity": 0,
        "objective": "binary:logistic",       
        "tree_method": "hist",       
              
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),        
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),       
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

   
    model = xgb.XGBClassifier(**params, enable_categorical=True)
    model.fit(X_train_, y_train_)
    predictions = model.predict(X_test_)
    roc = roc_auc_score(y_test_, predictions)
    return roc

In [99]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, timeout=600)

[I 2024-09-24 02:39:22,589] A new study created in memory with name: no-name-a6c4a53e-118b-48d5-8d2f-a5aae5b6a60a
[I 2024-09-24 02:39:26,255] Trial 0 finished with value: 0.9491319150358178 and parameters: {'lambda': 2.5314832139396193e-05, 'alpha': 1.3255191122465345e-08, 'subsample': 0.7202259262806232, 'colsample_bytree': 0.3677238520521517}. Best is trial 0 with value: 0.9491319150358178.
[I 2024-09-24 02:39:29,964] Trial 1 finished with value: 0.9491319150358178 and parameters: {'lambda': 6.111524218119611e-07, 'alpha': 2.9667419200197066e-05, 'subsample': 0.7338192687317644, 'colsample_bytree': 0.20531339733509243}. Best is trial 0 with value: 0.9491319150358178.
[I 2024-09-24 02:39:34,615] Trial 2 finished with value: 0.9521762965762703 and parameters: {'lambda': 0.0047991786273421265, 'alpha': 1.0714936485226588e-06, 'subsample': 0.2757805306044356, 'colsample_bytree': 0.43470853553051275}. Best is trial 2 with value: 0.9521762965762703.
[I 2024-09-24 02:39:38,420] Trial 3 fini

In [100]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  100
Best trial:
  Value: 0.9548564638262784
  Params: 
    lambda: 0.020032909628854265
    alpha: 6.033866130619256e-05
    subsample: 0.9603334490296908
    colsample_bytree: 0.9700098583859186


In [101]:
trial.params

{'lambda': 0.020032909628854265,
 'alpha': 6.033866130619256e-05,
 'subsample': 0.9603334490296908,
 'colsample_bytree': 0.9700098583859186}

In [102]:
model = xgb.XGBClassifier(**trial.params,  enable_categorical=True)
model.fit(X_train, y_train)

XGBClassifier(alpha=6.033866130619256e-05, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9700098583859186, device=None,
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=0.020032909628854265,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, ...)

In [103]:
final_predictions = model.predict(X_test)

In [104]:
final_roc = roc_auc_score(y_test, final_predictions)
final_roc

0.9555709684022029

In [106]:
model.save_model("model.json")

In [1]:
model

NameError: name 'model' is not defined

In [11]:
model = xgb.Booster()

In [12]:
model.load_model('model.json')

In [13]:
model.feature_names

['bank_id',
 'api_version',
 'connectivity_type',
 'total_duration',
 'authorized_duration']

In [15]:
test=xgb.DMatrix(processed[model.feature_names], enable_categorical=True)

In [16]:
model.predict(test)

array([9.9872059e-01, 9.3974656e-01, 9.9420989e-01, ..., 9.6131051e-01,
       9.9999940e-01, 2.9155487e-04], dtype=float32)

In [17]:
processed['status']

0         Executed
1         Executed
2         Executed
3         Executed
4         Executed
            ...   
499995    Executed
499996    Executed
499997    Executed
499998    Executed
499999      failed
Name: status, Length: 500000, dtype: object